In [1]:
import model.ieeg_cnn_rnn
import model.train

import processing.util as util
import keras

import os
import numpy as np
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# 1. Initialize the Model (CNN and CNN+LSTM)

In [2]:
imsize=32
numfreqs = 4
ieegdnn = model.ieeg_cnn_rnn.IEEGdnn(imsize=imsize, n_colors=numfreqs)

In [8]:
w_init = None
n_layers = (4,2,1)
poolsize = (2,2)
filtersize = (3,3)

size_fc = 1024
DROPOUT = False #True

# VGG-12 style later
vggcnn = ieegdnn._build_2dcnn(w_init=w_init, n_layers=n_layers, 
                              poolsize=poolsize, filter_size=filtersize)
vggcnn = ieegdnn._build_seq_output(vggcnn, size_fc, DROPOUT)
print(vggcnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 30, 30, 32)        1184      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 10, 10, 64)        18496     
__________

In [17]:
# load in CNN/LSTM
num_timewins = 5
size_mem = 128
size_fc = 1024
DROPOUT = False
cnn_lstm = ieegdnn.build_cnn_lstm(num_timewins=num_timewins, 
                                  size_mem=size_mem, size_fc=size_fc, DROPOUT=DROPOUT)

print(cnn_lstm.get_shape)

/Users/adam2392/Documents/dnn-unsupervised/dnn/model/ieeg_cnn_rnn.py:189: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  model.add(Merge(convnets, mode='concat'))


<bound method Tensor.get_shape of <tf.Tensor 'dense_17/truediv:0' shape=(?, 5, 2) dtype=float32>>


# 2. Load Image Data

In [4]:
# define data filepath to images
image_filepath = os.path.join('./traindata/finaltrain/trainimages.npy')
images = np.load(image_filepath)

# load the ylabeled data
ylabel_filepath = os.path.join('./traindata/finaltrain/trainlabels.npy')
ylabels = np.load(ylabel_filepath)

print(ylabels.shape)
print(images.shape)

(66169, 1)
(66169, 4, 32, 32)


In [6]:
# split into train,valid,test sets
datahandler = util.DataHandler()

# format the data correctly 
# (X_train, y_train), (X_val, y_val), (X_test, y_test) = datahandler.reformatinput(images, labels)
X_train, X_test, y_train, y_test = train_test_split(images, ylabels, test_size=0.33, random_state=42)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(type(X_train))
print(type(X_train[0,0,0,0]))
# print(X_val.shape)

(44333, 4, 32, 32)
(44333, 1)
(21836, 4, 32, 32)
<class 'numpy.ndarray'>
<class 'numpy.float32'>


# 3. Train CNN

In [19]:
loss = 'binary_crossentropy'
optimizer = keras.optimizers.Adam(lr=0.001, 
                                        beta_1=0.9, 
                                        beta_2=0.999,
                                        epsilon=1e-08,
                                        decay=0.0)
metrics = ['accuracy']
cnn_config = ieegdnn.compile_model(vggcnn, loss=loss, optimizer=optimizer, metrics=metrics)

In [21]:
display(cnn_config)

[{'class_name': 'InputLayer',
  'config': {'batch_input_shape': (None, 32, 32, 4),
   'dtype': 'float32',
   'name': 'input_6',
   'sparse': False}},
 {'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 32, 32, 4),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1),
   'dtype': 'float32',
   'filters': 32,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'kernel_size': (3, 3),
   'name': 'conv2d_36',
   'padding': 'valid',
   'strides': (1, 1),
   'trainable': True,
   'use_bias': True}},
 {'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None,

In [27]:
NUM_EPOCHS = 1
batch_size = 32 # or 64... or 24
# construct the image generator for data augmentation and construct
# the set of callbacks
aug = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
    height_shift_range=0.1, horizontal_flip=True,
    fill_mode="nearest")
callbacks = [keras.callbacks.LearningRateScheduler(model.train.poly_decay)]
INIT_LR = 5e-3
 
HH = vggcnn.fit_generator(
    aug.flow(X_train, y_train, batch_size=batch_size * G), # adds augmentation to data using generator
    validation_data=(X_test, y_test),  
    steps_per_epoch=len(X_train) // (batch_size * G),    #
    epochs=NUM_EPOCHS,
    callbacks=callbacks, verbose=2)

NameError: name 'X_train' is not defined

In [ ]:
import matplotlib.pyplot as plt
H = HH.history
print(HH.params)
print(H.keys())

# plot the training loss and accuracy
N = np.arange(0, len(H["loss"]))
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H["loss"], label="train_loss")
plt.plot(N, H["val_loss"], label="test_loss")
plt.plot(N, H["acc"], label="train_acc")
plt.plot(N, H["val_acc"], label="test_acc")
plt.title("VGG Net on MNIST")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

print(model.summary())

# 4. Use CNN to Train CNN-LSTM